
Email

This is an update on Dec C-NOS Actual Report. Please feel free to contact me if you have any questions.
 - As Japan MO total, Dec C-NOS devliered OO IYA with OOO $MM.
 - With this, 1H to Date C-NOS is OOO IYA with OOOO $MM which is -Opt vs. 1H target (OOO IYA).
     - [By Category] Over-delivery in Baby (+4% vs. 1H Target), Hair (+1%), under-delivery in Laundry (-9%), FE (-10%),   
                       Dish (-6%), Air (-14%),  Fem (-16%), Shave (-10%), Oral (-31%), Braun (-1%), POC (-40%).
     - [By Team] Under-delivery in Div1 (-8%), Div2 (-6%), D-com (-12%), Electro (-8%).
     - [By Top Customer] Over-delivery in Rakuten (+2%), TSURUHA (0%), under-delivery in WELCIA (-10%), CAINZ (-12%), MCC 
                          (-4%), SUGI (-6%), AEON (-5%), COSMOS (-21%), COSTCO (-18%), DONKI (-18%). 


Question
1. 1st Table 
  (SQL) Create SQL to show By Category 1H $MM, 1H IYA (Add Total)
  (Prompt) Write [By Category] part
2. 2nd Table
  (SQL) Create SQL to show By Team 1H $MM, 1H IYA (Add Total)
  (Prompt) Write [By team] part
3. 3rd Table
  (SQL) Create SQL to show By Top Customer 1H $MM, 1H IYA (Add Total)
  (Prompt) Write [By Top Customer] part
 

In [0]:
%sql
-- Category --

-- 中間テーブル(CTE)を作成し、FY2525 1H と 2025M07-12 を UNION する

WITH base_cnos AS (
  -- 中間テーブル：C-NOSだけを抽出しておく
  SELECT
    category,
    period,
    month,
    CAST(forecast_act_est AS INT) AS cnos,        -- 今年C-NOS（整数）
    CAST(ya_actual        AS INT) AS cnos_ya,     -- 昨年C-NOS（整数）
    CAST(target           AS INT) AS cnos_target  -- C-NOSターゲット（整数）
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE account = 'C-NOS'
),

fy2525_1h AS (
  -- FY2525 1H のカテゴリー別＋Total
  SELECT
    'FY2525 1H' AS term,                                 -- 期間ラベル
    CASE 
      WHEN GROUPING(category) = 1 THEN 'Total'           -- 合計行は 'Total'
      ELSE category
    END AS category,
    -- C-NOS IYA（今年 ÷ 昨年 × 100、小数点1桁）
    ROUND(
      CASE 
        WHEN SUM(cnos_ya) = 0 THEN NULL                  -- 昨年ゼロはNULL
        ELSE 100.0 * SUM(cnos) / SUM(cnos_ya)
      END,
      1
    ) AS cnos_iya,
    -- C-NOS $MM（今年）：値 / 150.1 / 1000 / 1000、小数点1桁
    ROUND(
      SUM(cnos) / 150.1 / 1000.0 / 1000.0,
      1
    ) AS cnos_mm,
    -- C-NOS Target $MM：ターゲット値を同じ換算、小数点1桁
    ROUND(
      SUM(cnos_target) / 150.1 / 1000.0 / 1000.0,
      1
    ) AS cnos_target_mm,
    -- vs Target %（ギャップ）：100 - (C-NOS_mm / Target_mm × 100)、小数1桁
    ROUND(
      CASE
        WHEN SUM(cnos_target) = 0 THEN NULL
        ELSE 100.0 - (100.0 * SUM(cnos) / SUM(cnos_target))
      END,
      1
    ) AS cnos_vs_target_pct
  FROM base_cnos
  WHERE period = 'FY2525 1H'
  GROUP BY
    GROUPING SETS (
      (category),   -- カテゴリー別
      ()            -- 全カテゴリー合計
    )
),

m0712 AS (
  -- 2025M07〜2025M12 のカテゴリー別＋Total
  SELECT
    '2025M07-12' AS term,                                -- 期間ラベル
    CASE 
      WHEN GROUPING(category) = 1 THEN 'Total'
      ELSE category
    END AS category,
    ROUND(
      CASE 
        WHEN SUM(cnos_ya) = 0 THEN NULL
        ELSE 100.0 * SUM(cnos) / SUM(cnos_ya)
      END,
      1
    ) AS cnos_iya,
    ROUND(
      SUM(cnos) / 150.1 / 1000.0 / 1000.0,
      1
    ) AS cnos_mm,
    ROUND(
      SUM(cnos_target) / 150.1 / 1000.0 / 1000.0,
      1
    ) AS cnos_target_mm,
    ROUND(
      CASE
        WHEN SUM(cnos_target) = 0 THEN NULL
        ELSE 100.0 - (100.0 * SUM(cnos) / SUM(cnos_target))
      END,
      1
    ) AS cnos_vs_target_pct
  FROM base_cnos
  WHERE month BETWEEN '2025M07' AND '2025M12'
  GROUP BY
    GROUPING SETS (
      (category),
      ()
    )
)

-- 2つの期間を UNION して1つのテーブルとして出力
SELECT
  term,
  category,
  cnos_iya,
  cnos_mm,
  cnos_target_mm,
  cnos_vs_target_pct
FROM fy2525_1h

UNION ALL

SELECT
  term,
  category,
  cnos_iya,
  cnos_mm,
  cnos_target_mm,
  cnos_vs_target_pct
FROM m0712

ORDER BY
  term,  -- まず期間ごとに並べる
  CASE 
    WHEN category = 'Total'           THEN 0
    WHEN category = 'Detergent'       THEN 1
    WHEN category = 'Fabric Enhancer' THEN 2
    WHEN category = 'Dish'            THEN 3
    WHEN category = 'Air'             THEN 4
    WHEN category = 'Baby'            THEN 5
    WHEN category = 'Fem'             THEN 6
    WHEN category = 'Hair'            THEN 7
    WHEN category = 'Shave'           THEN 8
    WHEN category = 'Oral'            THEN 9
    WHEN category = 'Braun'           THEN 10
    WHEN category = 'POC'             THEN 11
    ELSE 98
  END,
  category;

In [0]:
%sql

-- Team --

WITH base_cnos AS (
  -- ▼C-NOS のみを抽出し、金額は整数化しておく（Target も保持）
  SELECT
    org2,
    org3,
    period,
    month,
    CAST(forecast_act_est AS INT) AS cnos,        -- 今年 C-NOS（整数）
    CAST(ya_actual        AS INT) AS cnos_ya,     -- 昨年 C-NOS（整数）
    CAST(target           AS INT) AS cnos_target  -- C-NOS Target（整数）
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE account = 'C-NOS'
),

term_cnos AS (
  -- ▼対象期間に term ラベルを付与
  SELECT
    'FY2525 1H' AS term,
    org2,
    org3,
    cnos,
    cnos_ya,
    cnos_target
  FROM base_cnos
  WHERE period = 'FY2525 1H'

  UNION ALL

  SELECT
    '2025M07-12' AS term,
    org2,
    org3,
    cnos,
    cnos_ya,
    cnos_target
  FROM base_cnos
  WHERE month BETWEEN '2025M07' AND '2025M12'
),

summary AS (
  -- ▼各 term ごとに、Total / Div 2 / D-Com / Electro を一括で集計
  --   ・Div 2 は DP/MHC/WS（org3 ベース）の合計とみなす
  --   ・Div 1 は後で「残差」として計算する
  SELECT
    term,

    -- Total（全体）
    SUM(cnos)        AS total_cnos,
    SUM(cnos_ya)     AS total_cnos_ya,
    SUM(cnos_target) AS total_cnos_target,

    -- Div 2 = DP + MHC + WS（org3 ベース）
    SUM(CASE WHEN org3 IN (
          'Drug/Pharmacy Team',
          'Mass & Home Center Team',
          'Wholesale team'
        )
        THEN cnos END)        AS div2_cnos,
    SUM(CASE WHEN org3 IN (
          'Drug/Pharmacy Team',
          'Mass & Home Center Team',
          'Wholesale team'
        )
        THEN cnos_ya END)     AS div2_cnos_ya,
    SUM(CASE WHEN org3 IN (
          'Drug/Pharmacy Team',
          'Mass & Home Center Team',
          'Wholesale team'
        )
        THEN cnos_target END) AS div2_cnos_target,

    -- D-Com（org2 ベース）
    SUM(CASE WHEN org2 = 'D-Com'
        THEN cnos END)        AS dcom_cnos,
    SUM(CASE WHEN org2 = 'D-Com'
        THEN cnos_ya END)     AS dcom_cnos_ya,
    SUM(CASE WHEN org2 = 'D-Com'
        THEN cnos_target END) AS dcom_cnos_target,

    -- Electro（org2 ベース）
    SUM(CASE WHEN org2 = 'Electro'
        THEN cnos END)        AS electro_cnos,
    SUM(CASE WHEN org2 = 'Electro'
        THEN cnos_ya END)     AS electro_cnos_ya,
    SUM(CASE WHEN org2 = 'Electro'
        THEN cnos_target END) AS electro_cnos_target
  FROM term_cnos
  GROUP BY term
),

detail AS (
  -- ▼DP / MHC / WS / D-Com / Electro の個別行（内訳）を集計
  --   ※ D-Com / Electro は summary でも使っているが、
  --     ここでは「行として出すため」に再度集計している
  SELECT
    term,
    'DP' AS team,
    SUM(cnos)        AS sum_cnos,
    SUM(cnos_ya)     AS sum_cnos_ya,
    SUM(cnos_target) AS sum_cnos_target
  FROM term_cnos
  WHERE org3 = 'Drug/Pharmacy Team'
  GROUP BY term

  UNION ALL

  SELECT
    term,
    'MHC' AS team,
    SUM(cnos)        AS sum_cnos,
    SUM(cnos_ya)     AS sum_cnos_ya,
    SUM(cnos_target) AS sum_cnos_target
  FROM term_cnos
  WHERE org3 = 'Mass & Home Center Team'
  GROUP BY term

  UNION ALL

  SELECT
    term,
    'WS' AS team,
    SUM(cnos)        AS sum_cnos,
    SUM(cnos_ya)     AS sum_cnos_ya,
    SUM(cnos_target) AS sum_cnos_target
  FROM term_cnos
  WHERE org3 = 'Wholesale team'
  GROUP BY term

  UNION ALL

  SELECT
    term,
    'D-Com' AS team,
    SUM(cnos)        AS sum_cnos,
    SUM(cnos_ya)     AS sum_cnos_ya,
    SUM(cnos_target) AS sum_cnos_target
  FROM term_cnos
  WHERE org2 = 'D-Com'
  GROUP BY term

  UNION ALL

  SELECT
    term,
    'Electro' AS team,
    SUM(cnos)        AS sum_cnos,
    SUM(cnos_ya)     AS sum_cnos_ya,
    SUM(cnos_target) AS sum_cnos_target
  FROM term_cnos
  WHERE org2 = 'Electro'
  GROUP BY term
),

node_agg AS (
  -- ▼Total / Div 1 / Div 2 と detail（DP/MHC/WS/D-Com/Electro）を一つのテーブルにまとめる
  --   Div 1 は「Total - Div2 - D-Com - Electro」で算出（残差：今年・昨年・Target）
  SELECT
    term,
    'Total' AS team,
    total_cnos         AS sum_cnos,
    total_cnos_ya      AS sum_cnos_ya,
    total_cnos_target  AS sum_cnos_target
  FROM summary

  UNION ALL

  SELECT
    term,
    'Div 2' AS team,
    div2_cnos         AS sum_cnos,
    div2_cnos_ya      AS sum_cnos_ya,
    div2_cnos_target  AS sum_cnos_target
  FROM summary

  UNION ALL

  SELECT
    term,
    'Div 1' AS team,
    -- Div 1 = Total - Div 2 - D-Com - Electro（今年分）
    (total_cnos
     - COALESCE(div2_cnos, 0)
     - COALESCE(dcom_cnos, 0)
     - COALESCE(electro_cnos, 0)
    ) AS sum_cnos,
    -- Div 1 = Total - Div 2 - D-Com - Electro（昨年分）
    (total_cnos_ya
     - COALESCE(div2_cnos_ya, 0)
     - COALESCE(dcom_cnos_ya, 0)
     - COALESCE(electro_cnos_ya, 0)
    ) AS sum_cnos_ya,
    -- Div 1 Target = Total Target - Div 2 Target - D-Com Target - Electro Target
    (total_cnos_target
     - COALESCE(div2_cnos_target, 0)
     - COALESCE(dcom_cnos_target, 0)
     - COALESCE(electro_cnos_target, 0)
    ) AS sum_cnos_target
  FROM summary

  UNION ALL

  -- DP / MHC / WS / D-Com / Electro の個別行
  SELECT
    term,
    team,
    sum_cnos,
    sum_cnos_ya,
    sum_cnos_target
  FROM detail
)

-- ▼最終出力：IYA・$MM・Target $MM・vs Target % を計算し、指定順に並べる
SELECT
  term,
  team,
  -- C-NOS IYA：今年 ÷ 昨年 × 100（小数第1位）
  ROUND(
    CASE
      WHEN sum_cnos_ya IS NULL OR sum_cnos_ya = 0 THEN NULL
      ELSE 100.0 * sum_cnos / sum_cnos_ya
    END,
    1
  ) AS cnos_iya,
  -- C-NOS $MM：cnos / 150.1 / 1000 / 1000（小数第1位）
  ROUND(
    CASE
      WHEN sum_cnos IS NULL THEN NULL
      ELSE sum_cnos / 150.1 / 1000.0 / 1000.0
    END,
    1
  ) AS cnos_mm,
  -- C-NOS Target $MM：target / 150.1 / 1000 / 1000（小数第1位）
  ROUND(
    CASE
      WHEN sum_cnos_target IS NULL THEN NULL
      ELSE sum_cnos_target / 150.1 / 1000.0 / 1000.0
    END,
    1
  ) AS cnos_target_mm,
  -- vs Target %（ギャップ）：100 - (C-NOS / Target × 100)、小数第1位
  ROUND(
    CASE
      WHEN sum_cnos_target IS NULL OR sum_cnos_target = 0 THEN NULL
      ELSE 100.0 - (100.0 * sum_cnos / sum_cnos_target)
    END,
    1
  ) AS cnos_vs_target_pct
FROM node_agg
ORDER BY
  term,
  CASE
    -- 表示順：Total → Div 1 → Div 2 → DP → MHC → WS → D-Com → Electro
    WHEN team = 'Total'   THEN 1
    WHEN team = 'Div 1'   THEN 2
    WHEN team = 'Div 2'   THEN 3
    WHEN team = 'DP'      THEN 4
    WHEN team = 'MHC'     THEN 5
    WHEN team = 'WS'      THEN 6
    WHEN team = 'D-Com'   THEN 7
    WHEN team = 'Electro' THEN 8
    ELSE 99
  END,
  team;

In [0]:
%sql
-- ------------------------------------------------------------
-- 粒度：
--   term × customer_label
-- customer_label には以下を縦に並べる（同じ列）：
--   Div 1（17社合計）
--   Welcia, Cainz, ..., Sunday（各社）
--   D-Com（Amazon+Rakuten合計）
--   Amazon, Rakuten（各社）
--
-- 指標：
--   C-NOS IYA（今年÷昨年×100, 小数1桁）
--   C-NOS $MM（今年 C-NOS/150.1/1000/1000, 小数1桁）
--   C-NOS Target $MM
--   vs Target % = 100 - (C-NOS / C-NOS Target × 100)（小数1桁）
--
-- 前提：
--   Div 1 = org2 = 'Division 1'
--   D-Com = org2 = 'D-Com'
-- ------------------------------------------------------------

WITH base_cnos AS (
  -- ▼C-NOS のみを抽出し、金額を整数化（Target も保持）
  SELECT
    org2,
    customer,
    period,
    month,
    CAST(forecast_act_est AS INT) AS cnos,        -- 今年 C-NOS（整数）
    CAST(ya_actual        AS INT) AS cnos_ya,     -- 昨年 C-NOS（整数）
    CAST(target           AS INT) AS cnos_target  -- C-NOS Target（整数）
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE account = 'C-NOS'
),

term_cnos AS (
  -- ▼対象期間に term ラベルを付与
  SELECT
    'FY2525 1H' AS term,
    org2,
    customer,
    cnos,
    cnos_ya,
    cnos_target
  FROM base_cnos
  WHERE period = 'FY2525 1H'

  UNION ALL

  SELECT
    '2025M07-12' AS term,
    org2,
    customer,
    cnos,
    cnos_ya,
    cnos_target
  FROM base_cnos
  WHERE month BETWEEN '2025M07' AND '2025M12'
),

filtered AS (
  -- ▼org2 と customer から Div 1 / D-Com を判定し、対象顧客だけに絞り込み
  SELECT
    term,
    CASE
      -- Div 1 顧客（org2 = Division 1）
      WHEN org2 = 'Division 1' AND customer IN (
        'FN-WELCIA YAKKYOKU',
        'FN-CAINZ(C003)',
        'FN-MATSUKIYO_COCOKARA (B008)',
        'FN-SUGI_YAKKYOKU(I015)',
        'FN-AEON_GROUP(JUSCOJ4)(C005)',
        'FN-COSMOS_YAKUHIN(W030)',
        'FN-COSTCO(A062)',
        'FN-TSURUHA_HOKK+TOHO(E030)',
        'FN-DONKIHOTE(A094)',
        'FN-SUN_DRUG(D005)',
        'FN-TGN',
        'FN-KYORINDO(H011)',
        'FN-LADY_YAKKYOKU(K067)',
        'FN-KUSURINO FUKUTARO (G043)',
        'FN-ELEVEN(L002)',
        'FN-UNY(B009)',
        'FN-SUNDAY(E015)'
      ) THEN 'Div 1'
      -- D-Com 顧客（org2 = D-Com）
      WHEN org2 = 'D-Com' AND customer IN (
        'FN-AMAZON JAPAN',
        'FN-RAKUTEN'
      ) THEN 'D-Com'
      ELSE NULL
    END AS division,          -- Div 1 / D-Com
    customer,
    cnos,
    cnos_ya,
    cnos_target
  FROM term_cnos
  WHERE customer IN (
    -- Div 1 顧客
    'FN-WELCIA YAKKYOKU',
    'FN-CAINZ(C003)',
    'FN-MATSUKIYO_COCOKARA (B008)',
    'FN-SUGI_YAKKYOKU(I015)',
    'FN-AEON_GROUP(JUSCOJ4)(C005)',
    'FN-COSMOS_YAKUHIN(W030)',
    'FN-COSTCO(A062)',
    'FN-TSURUHA_HOKK+TOHO(E030)',
    'FN-DONKIHOTE(A094)',
    'FN-SUN_DRUG(D005)',
    'FN-TGN',
    'FN-KYORINDO(H011)',
    'FN-LADY_YAKKYOKU(K067)',
    'FN-KUSURINO FUKUTARO (G043)',
    'FN-ELEVEN(L002)',
    'FN-UNY(B009)',
    'FN-SUNDAY(E015)',
    -- D-Com 顧客
    'FN-AMAZON JAPAN',
    'FN-RAKUTEN'
  )
),

cust_agg AS (
  -- ▼term × division × customer ごとに C-NOS を集計（顧客粒度）
  SELECT
    term,
    division,
    customer,
    SUM(cnos)        AS sum_cnos,
    SUM(cnos_ya)     AS sum_cnos_ya,
    SUM(cnos_target) AS sum_cnos_target
  FROM filtered
  WHERE division IS NOT NULL
  GROUP BY
    term,
    division,
    customer
),

div_agg AS (
  -- ▼Div 1 / D-Com の合計行（17社合計 / 2社合計）
  SELECT
    term,
    division,
    SUM(sum_cnos)        AS sum_cnos,
    SUM(sum_cnos_ya)     AS sum_cnos_ya,
    SUM(sum_cnos_target) AS sum_cnos_target
  FROM cust_agg
  GROUP BY
    term,
    division
),

labeled AS (
  -- ▼Div 行と顧客行を「同じ列（customer_label）」に並べる
  SELECT
    term,
    -- Div 1・D-Com の合計行
    CASE
      WHEN division = 'Div 1' THEN 'Div 1'
      WHEN division = 'D-Com' THEN 'D-Com'
      ELSE division
    END AS customer_label,
    sum_cnos,
    sum_cnos_ya,
    sum_cnos_target
  FROM div_agg

  UNION ALL

  SELECT
    term,
    -- 顧客行：短い名前に変換（インデント用に先頭半角スペース）
    CASE customer
      WHEN 'FN-WELCIA YAKKYOKU'                 THEN ' Welcia'
      WHEN 'FN-CAINZ(C003)'                     THEN ' Cainz'
      WHEN 'FN-MATSUKIYO_COCOKARA (B008)'       THEN ' MCC'
      WHEN 'FN-SUGI_YAKKYOKU(I015)'             THEN ' Sugi'
      WHEN 'FN-AEON_GROUP(JUSCOJ4)(C005)'       THEN ' Aeon'
      WHEN 'FN-COSMOS_YAKUHIN(W030)'            THEN ' Cosmos'
      WHEN 'FN-COSTCO(A062)'                    THEN ' Costco'
      WHEN 'FN-TSURUHA_HOKK+TOHO(E030)'         THEN ' Tsuruha'
      WHEN 'FN-DONKIHOTE(A094)'                 THEN ' Donki'
      WHEN 'FN-SUN_DRUG(D005)'                  THEN ' Sundrug'
      WHEN 'FN-TGN'                             THEN ' TGN'
      WHEN 'FN-KYORINDO(H011)'                  THEN ' Kyorindo'
      WHEN 'FN-LADY_YAKKYOKU(K067)'             THEN ' Lady'
      WHEN 'FN-KUSURINO FUKUTARO (G043)'        THEN ' Fukutaro'
      WHEN 'FN-ELEVEN(L002)'                    THEN ' Eleven'
      WHEN 'FN-UNY(B009)'                       THEN ' Uny'
      WHEN 'FN-SUNDAY(E015)'                    THEN ' Sunday'
      WHEN 'FN-AMAZON JAPAN'                    THEN ' Amazon'
      WHEN 'FN-RAKUTEN'                         THEN ' Rakuten'
      ELSE customer
    END AS customer_label,
    sum_cnos,
    sum_cnos_ya,
    sum_cnos_target
  FROM cust_agg
)

-- ▼最終出力：customer_label 1列で、Div1/顧客/D-Com/顧客を縦に並べる
SELECT
  term,
  customer_label AS customer,
  -- C-NOS IYA：今年 ÷ 昨年 × 100（小数第1位）
  ROUND(
    CASE
      WHEN sum_cnos_ya = 0 THEN NULL
      ELSE 100.0 * sum_cnos / sum_cnos_ya
    END,
    1
  ) AS cnos_iya,
  -- C-NOS $MM：cnos / 150.1 / 1000 / 1000（小数第1位）
  ROUND(
    sum_cnos / 150.1 / 1000.0 / 1000.0,
    1
  ) AS cnos_mm,
  -- C-NOS Target $MM：target / 150.1 / 1000 / 1000（小数第1位）
  ROUND(
    CASE
      WHEN sum_cnos_target = 0 THEN NULL
      ELSE sum_cnos_target / 150.1 / 1000.0 / 1000.0
    END,
    1
  ) AS cnos_target_mm,
  -- vs Target %（ギャップ）：100 - (C-NOS / Target × 100)、小数第1位
  ROUND(
    CASE
      WHEN sum_cnos_target = 0 THEN NULL
      ELSE 100.0 - (100.0 * sum_cnos / sum_cnos_target)
    END,
    1
  ) AS cnos_vs_target_pct
FROM labeled
ORDER BY
  term,
  CASE
    -- 表示順：Div 1 → Div 1 顧客 → D-Com → D-Com 顧客
    WHEN customer = 'Div 1'    THEN 1
    WHEN customer = ' Welcia'  THEN 2
    WHEN customer = ' Cainz'   THEN 3
    WHEN customer = ' MCC'     THEN 4
    WHEN customer = ' Sugi'    THEN 5
    WHEN customer = ' Aeon'    THEN 6
    WHEN customer = ' Cosmos'  THEN 7
    WHEN customer = ' Costco'  THEN 8
    WHEN customer = ' Tsuruha' THEN 9
    WHEN customer = ' Donki'   THEN 10
    WHEN customer = ' Sundrug' THEN 11
    WHEN customer = ' TGN'     THEN 12
    WHEN customer = ' Kyorindo'THEN 13
    WHEN customer = ' Lady'    THEN 14
    WHEN customer = ' Fukutaro'THEN 15
    WHEN customer = ' Eleven'  THEN 16
    WHEN customer = ' Uny'     THEN 17
    WHEN customer = ' Sunday'  THEN 18
    WHEN customer = 'D-Com'    THEN 19
    WHEN customer = ' Amazon'  THEN 20
    WHEN customer = ' Rakuten' THEN 21
    ELSE 99
  END,
  customer;

{
  "sections": [
    {
      "section": "C-NOS Overview",
      "description": "This section provides a key C-NOS highlights of C-NOS",
      "items": [
        {
          "simple_question": "Is our C-NOS on target by Category?",
          "sql_query": "-- Category --\n\n-- 中間テーブル(CTE)を作成し、FY2525 1H と 2025M07-12 を UNION する\n\nWITH base_cnos AS (\n  -- 中間テーブル：C-NOSだけを抽出しておく\n  SELECT\n    category,\n    period,\n    month,\n    CAST(forecast_act_est AS INT) AS cnos,        -- 今年C-NOS（整数）\n    CAST(ya_actual        AS INT) AS cnos_ya,     -- 昨年C-NOS（整数）\n    CAST(target           AS INT) AS cnos_target  -- C-NOSターゲット（整数）\n  FROM hive_metastore.japan_gold.cnos_vw\n  WHERE account = 'C-NOS'\n),\n\nfy2525_1h AS (\n  -- FY2525 1H のカテゴリー別＋Total\n  SELECT\n    'FY2525 1H' AS term,                                 -- 期間ラベル\n    CASE \n      WHEN GROUPING(category) = 1 THEN 'Total'           -- 合計行は 'Total'\n      ELSE category\n    END AS category,\n    -- C-NOS IYA（今年 ÷ 昨年 × 100、小数点1桁）\n    ROUND(\n      CASE \n        WHEN SUM(cnos_ya) = 0 THEN NULL                  -- 昨年ゼロはNULL\n        ELSE 100.0 * SUM(cnos) / SUM(cnos_ya)\n      END,\n      1\n    ) AS cnos_iya,\n    -- C-NOS $MM（今年）：値 / 150.1 / 1000 / 1000、小数点1桁\n    ROUND(\n      SUM(cnos) / 150.1 / 1000.0 / 1000.0,\n      1\n    ) AS cnos_mm,\n    -- C-NOS Target $MM：ターゲット値を同じ換算、小数点1桁\n    ROUND(\n      SUM(cnos_target) / 150.1 / 1000.0 / 1000.0,\n      1\n    ) AS cnos_target_mm,\n    -- vs Target %（ギャップ）：100 - (C-NOS_mm / Target_mm × 100)、小数1桁\n    ROUND(\n      CASE\n        WHEN SUM(cnos_target) = 0 THEN NULL\n        ELSE 100.0 - (100.0 * SUM(cnos) / SUM(cnos_target))\n      END,\n      1\n    ) AS cnos_vs_target_pct\n  FROM base_cnos\n  WHERE period = 'FY2525 1H'\n  GROUP BY\n    GROUPING SETS (\n      (category),   -- カテゴリー別\n      ()            -- 全カテゴリー合計\n    )\n),\n\nm0712 AS (\n  -- 2025M07〜2025M12 のカテゴリー別＋Total\n  SELECT\n    '2025M07-12' AS term,                                -- 期間ラベル\n    CASE \n      WHEN GROUPING(category) = 1 THEN 'Total'\n      ELSE category\n    END AS category,\n    ROUND(\n      CASE \n        WHEN SUM(cnos_ya) = 0 THEN NULL\n        ELSE 100.0 * SUM(cnos) / SUM(cnos_ya)\n      END,\n      1\n    ) AS cnos_iya,\n    ROUND(\n      SUM(cnos) / 150.1 / 1000.0 / 1000.0,\n      1\n    ) AS cnos_mm,\n    ROUND(\n      SUM(cnos_target) / 150.1 / 1000.0 / 1000.0,\n      1\n    ) AS cnos_target_mm,\n    ROUND(\n      CASE\n        WHEN SUM(cnos_target) = 0 THEN NULL\n        ELSE 100.0 - (100.0 * SUM(cnos) / SUM(cnos_target))\n      END,\n      1\n    ) AS cnos_vs_target_pct\n  FROM base_cnos\n  WHERE month BETWEEN '2025M07' AND '2025M12'\n  GROUP BY\n    GROUPING SETS (\n      (category),\n      ()\n    )\n)\n\n-- 2つの期間を UNION して1つのテーブルとして出力\nSELECT\n  term,\n  category,\n  cnos_iya,\n  cnos_mm,\n  cnos_target_mm,\n  cnos_vs_target_pct\nFROM fy2525_1h\n\nUNION ALL\n\nSELECT\n  term,\n  category,\n  cnos_iya,\n  cnos_mm,\n  cnos_target_mm,\n  cnos_vs_target_pct\nFROM m0712\n\nORDER BY\n  term,  -- まず期間ごとに並べる\n  CASE \n    WHEN category = 'Total'           THEN 0\n    WHEN category = 'Detergent'       THEN 1\n    WHEN category = 'Fabric Enhancer' THEN 2\n    WHEN category = 'Dish'            THEN 3\n    WHEN category = 'Air'             THEN 4\n    WHEN category = 'Baby'            THEN 5\n    WHEN category = 'Fem'             THEN 6\n    WHEN category = 'Hair'            THEN 7\n    WHEN category = 'Shave'           THEN 8\n    WHEN category = 'Oral'            THEN 9\n    WHEN category = 'Braun'           THEN 10\n    WHEN category = 'POC'             THEN 11\n    ELSE 98\n  END,\n  category;",
          "reference": "Your analysis must strictly follow the following format. Do not add text nor line by yourself.\n\n- As Japan MO total, Oct C-NOS delivered XXX IYA with XXX $MM.\n- With this, 1H To Date C-NOS is XXX IYA with XXX $MM which is -Xpt vs. 1H target (XXX IYA)\n      - By Category: Over-delivery in Category A (+X% vs. 1H Target), Category B (+X%), ..., under-delivery in Laundry (-X%), ... .\n    *This Category A, B should show the correct category name in the table\n        - By Team: Over-delivery in D-Com (+X%), under-deliery in Team A (-X%), ...\n    *This Team A, B should show the correct category name in the table\n        - By Customer: Over-delivery in Customer (+X%), TSURUYA (+X%), under-delivery in Customer C (-X%),...\n    *This Customer A, B should show the correct category name in the table\n - If it's asked IYA, the numbers should be between 0~100 \n - If go over target, use Green color and underline to highlight for numbers. For customer & category, make it bold and colored\n  - If below target, use Red color and underline to highlight for numbers. For customer & category, make it bold and colored. \n\nAfter that, please provide the 1-2 analysis where is the opportunity in customer, team, category.\n\nThen, Present a rich interactive HTML table with all boarders. The order of the table should be the same as provided. Make the table font 0.75 times smaller\nAvailable data: {}",
          "number_of_plots": 0
        },
        {
          "simple_question": "Is our C-NOS on target by Team?",
          "sql_query": "WITH base_cnos AS (\n  -- ▼C-NOS のみを抽出し、金額は整数化しておく（Target も保持）\n  SELECT\n    org2,\n    org3,\n    period,\n    month,\n    CAST(forecast_act_est AS INT) AS cnos,        -- 今年 C-NOS（整数）\n    CAST(ya_actual        AS INT) AS cnos_ya,     -- 昨年 C-NOS（整数）\n    CAST(target           AS INT) AS cnos_target  -- C-NOS Target（整数）\n  FROM hive_metastore.japan_gold.cnos_vw\n  WHERE account = 'C-NOS'\n),\n\nterm_cnos AS (\n  -- ▼対象期間に term ラベルを付与\n  SELECT\n    'FY2525 1H' AS term,\n    org2,\n    org3,\n    cnos,\n    cnos_ya,\n    cnos_target\n  FROM base_cnos\n  WHERE period = 'FY2525 1H'\n\n  UNION ALL\n\n  SELECT\n    '2025M07-12' AS term,\n    org2,\n    org3,\n    cnos,\n    cnos_ya,\n    cnos_target\n  FROM base_cnos\n  WHERE month BETWEEN '2025M07' AND '2025M12'\n),\n\nsummary AS (\n  -- ▼各 term ごとに、Total / Div 2 / D-Com / Electro を一括で集計\n  --   ・Div 2 は DP/MHC/WS（org3 ベース）の合計とみなす\n  --   ・Div 1 は後で「残差」として計算する\n  SELECT\n    term,\n\n    -- Total（全体）\n    SUM(cnos)        AS total_cnos,\n    SUM(cnos_ya)     AS total_cnos_ya,\n    SUM(cnos_target) AS total_cnos_target,\n\n    -- Div 2 = DP + MHC + WS（org3 ベース）\n    SUM(CASE WHEN org3 IN (\n          'Drug/Pharmacy Team',\n          'Mass & Home Center Team',\n          'Wholesale team'\n        )\n        THEN cnos END)        AS div2_cnos,\n    SUM(CASE WHEN org3 IN (\n          'Drug/Pharmacy Team',\n          'Mass & Home Center Team',\n          'Wholesale team'\n        )\n        THEN cnos_ya END)     AS div2_cnos_ya,\n    SUM(CASE WHEN org3 IN (\n          'Drug/Pharmacy Team',\n          'Mass & Home Center Team',\n          'Wholesale team'\n        )\n        THEN cnos_target END) AS div2_cnos_target,\n\n    -- D-Com（org2 ベース）\n    SUM(CASE WHEN org2 = 'D-Com'\n        THEN cnos END)        AS dcom_cnos,\n    SUM(CASE WHEN org2 = 'D-Com'\n        THEN cnos_ya END)     AS dcom_cnos_ya,\n    SUM(CASE WHEN org2 = 'D-Com'\n        THEN cnos_target END) AS dcom_cnos_target,\n\n    -- Electro（org2 ベース）\n    SUM(CASE WHEN org2 = 'Electro'\n        THEN cnos END)        AS electro_cnos,\n    SUM(CASE WHEN org2 = 'Electro'\n        THEN cnos_ya END)     AS electro_cnos_ya,\n    SUM(CASE WHEN org2 = 'Electro'\n        THEN cnos_target END) AS electro_cnos_target\n  FROM term_cnos\n  GROUP BY term\n),\n\ndetail AS (\n  -- ▼DP / MHC / WS / D-Com / Electro の個別行（内訳）を集計\n  --   ※ D-Com / Electro は summary でも使っているが、\n  --     ここでは「行として出すため」に再度集計している\n  SELECT\n    term,\n    'DP' AS team,\n    SUM(cnos)        AS sum_cnos,\n    SUM(cnos_ya)     AS sum_cnos_ya,\n    SUM(cnos_target) AS sum_cnos_target\n  FROM term_cnos\n  WHERE org3 = 'Drug/Pharmacy Team'\n  GROUP BY term\n\n  UNION ALL\n\n  SELECT\n    term,\n    'MHC' AS team,\n    SUM(cnos)        AS sum_cnos,\n    SUM(cnos_ya)     AS sum_cnos_ya,\n    SUM(cnos_target) AS sum_cnos_target\n  FROM term_cnos\n  WHERE org3 = 'Mass & Home Center Team'\n  GROUP BY term\n\n  UNION ALL\n\n  SELECT\n    term,\n    'WS' AS team,\n    SUM(cnos)        AS sum_cnos,\n    SUM(cnos_ya)     AS sum_cnos_ya,\n    SUM(cnos_target) AS sum_cnos_target\n  FROM term_cnos\n  WHERE org3 = 'Wholesale team'\n  GROUP BY term\n\n  UNION ALL\n\n  SELECT\n    term,\n    'D-Com' AS team,\n    SUM(cnos)        AS sum_cnos,\n    SUM(cnos_ya)     AS sum_cnos_ya,\n    SUM(cnos_target) AS sum_cnos_target\n  FROM term_cnos\n  WHERE org2 = 'D-Com'\n  GROUP BY term\n\n  UNION ALL\n\n  SELECT\n    term,\n    'Electro' AS team,\n    SUM(cnos)        AS sum_cnos,\n    SUM(cnos_ya)     AS sum_cnos_ya,\n    SUM(cnos_target) AS sum_cnos_target\n  FROM term_cnos\n  WHERE org2 = 'Electro'\n  GROUP BY term\n),\n\nnode_agg AS (\n  -- ▼Total / Div 1 / Div 2 と detail（DP/MHC/WS/D-Com/Electro）を一つのテーブルにまとめる\n  --   Div 1 は「Total - Div2 - D-Com - Electro」で算出（残差：今年・昨年・Target）\n  SELECT\n    term,\n    'Total' AS team,\n    total_cnos         AS sum_cnos,\n    total_cnos_ya      AS sum_cnos_ya,\n    total_cnos_target  AS sum_cnos_target\n  FROM summary\n\n  UNION ALL\n\n  SELECT\n    term,\n    'Div 2' AS team,\n    div2_cnos         AS sum_cnos,\n    div2_cnos_ya      AS sum_cnos_ya,\n    div2_cnos_target  AS sum_cnos_target\n  FROM summary\n\n  UNION ALL\n\n  SELECT\n    term,\n    'Div 1' AS team,\n    -- Div 1 = Total - Div 2 - D-Com - Electro（今年分）\n    (total_cnos\n     - COALESCE(div2_cnos, 0)\n     - COALESCE(dcom_cnos, 0)\n     - COALESCE(electro_cnos, 0)\n    ) AS sum_cnos,\n    -- Div 1 = Total - Div 2 - D-Com - Electro（昨年分）\n    (total_cnos_ya\n     - COALESCE(div2_cnos_ya, 0)\n     - COALESCE(dcom_cnos_ya, 0)\n     - COALESCE(electro_cnos_ya, 0)\n    ) AS sum_cnos_ya,\n    -- Div 1 Target = Total Target - Div 2 Target - D-Com Target - Electro Target\n    (total_cnos_target\n     - COALESCE(div2_cnos_target, 0)\n     - COALESCE(dcom_cnos_target, 0)\n     - COALESCE(electro_cnos_target, 0)\n    ) AS sum_cnos_target\n  FROM summary\n\n  UNION ALL\n\n  -- DP / MHC / WS / D-Com / Electro の個別行\n  SELECT\n    term,\n    team,\n    sum_cnos,\n    sum_cnos_ya,\n    sum_cnos_target\n  FROM detail\n)\n\n-- ▼最終出力：IYA・$MM・Target $MM・vs Target % を計算し、指定順に並べる\nSELECT\n  term,\n  team,\n  -- C-NOS IYA：今年 ÷ 昨年 × 100（小数第1位）\n  ROUND(\n    CASE\n      WHEN sum_cnos_ya IS NULL OR sum_cnos_ya = 0 THEN NULL\n      ELSE 100.0 * sum_cnos / sum_cnos_ya\n    END,\n    1\n  ) AS cnos_iya,\n  -- C-NOS $MM：cnos / 150.1 / 1000 / 1000（小数第1位）\n  ROUND(\n    CASE\n      WHEN sum_cnos IS NULL THEN NULL\n      ELSE sum_cnos / 150.1 / 1000.0 / 1000.0\n    END,\n    1\n  ) AS cnos_mm,\n  -- C-NOS Target $MM：target / 150.1 / 1000 / 1000（小数第1位）\n  ROUND(\n    CASE\n      WHEN sum_cnos_target IS NULL THEN NULL\n      ELSE sum_cnos_target / 150.1 / 1000.0 / 1000.0\n    END,\n    1\n  ) AS cnos_target_mm,\n  -- vs Target %（ギャップ）：100 - (C-NOS / Target × 100)、小数第1位\n  ROUND(\n    CASE\n      WHEN sum_cnos_target IS NULL OR sum_cnos_target = 0 THEN NULL\n      ELSE 100.0 - (100.0 * sum_cnos / sum_cnos_target)\n    END,\n    1\n  ) AS cnos_vs_target_pct\nFROM node_agg\nORDER BY\n  term,\n  CASE\n    -- 表示順：Total → Div 1 → Div 2 → DP → MHC → WS → D-Com → Electro\n    WHEN team = 'Total'   THEN 1\n    WHEN team = 'Div 1'   THEN 2\n    WHEN team = 'Div 2'   THEN 3\n    WHEN team = 'DP'      THEN 4\n    WHEN team = 'MHC'     THEN 5\n    WHEN team = 'WS'      THEN 6\n    WHEN team = 'D-Com'   THEN 7\n    WHEN team = 'Electro' THEN 8\n    ELSE 99\n  END,\n  team;",
          "reference": "Present a rich interactive HTML table with all boarders. The order of the table should be the same as provided. Make the table font 0.75 times smaller\nAvailable data: {}",
          "number_of_plots": 0
        },
        {
          "simple_question": "Is our C-NOS on target by Customer?",
          "sql_query": "-- ------------------------------------------------------------\n-- 粒度：\n--   term × customer_label\n-- customer_label には以下を縦に並べる（同じ列）：\n--   Div 1（17社合計）\n--   Welcia, Cainz, ..., Sunday（各社）\n--   D-Com（Amazon+Rakuten合計）\n--   Amazon, Rakuten（各社）\n--\n-- 指標：\n--   C-NOS IYA（今年÷昨年×100, 小数1桁）\n--   C-NOS $MM（今年 C-NOS/150.1/1000/1000, 小数1桁）\n--   C-NOS Target $MM\n--   vs Target % = 100 - (C-NOS / C-NOS Target × 100)（小数1桁）\n--\n-- 前提：\n--   Div 1 = org2 = 'Division 1'\n--   D-Com = org2 = 'D-Com'\n-- ------------------------------------------------------------\n\nWITH base_cnos AS (\n  -- ▼C-NOS のみを抽出し、金額を整数化（Target も保持）\n  SELECT\n    org2,\n    customer,\n    period,\n    month,\n    CAST(forecast_act_est AS INT) AS cnos,        -- 今年 C-NOS（整数）\n    CAST(ya_actual        AS INT) AS cnos_ya,     -- 昨年 C-NOS（整数）\n    CAST(target           AS INT) AS cnos_target  -- C-NOS Target（整数）\n  FROM hive_metastore.japan_gold.cnos_vw\n  WHERE account = 'C-NOS'\n),\n\nterm_cnos AS (\n  -- ▼対象期間に term ラベルを付与\n  SELECT\n    'FY2525 1H' AS term,\n    org2,\n    customer,\n    cnos,\n    cnos_ya,\n    cnos_target\n  FROM base_cnos\n  WHERE period = 'FY2525 1H'\n\n  UNION ALL\n\n  SELECT\n    '2025M07-12' AS term,\n    org2,\n    customer,\n    cnos,\n    cnos_ya,\n    cnos_target\n  FROM base_cnos\n  WHERE month BETWEEN '2025M07' AND '2025M12'\n),\n\nfiltered AS (\n  -- ▼org2 と customer から Div 1 / D-Com を判定し、対象顧客だけに絞り込み\n  SELECT\n    term,\n    CASE\n      -- Div 1 顧客（org2 = Division 1）\n      WHEN org2 = 'Division 1' AND customer IN (\n        'FN-WELCIA YAKKYOKU',\n        'FN-CAINZ(C003)',\n        'FN-MATSUKIYO_COCOKARA (B008)',\n        'FN-SUGI_YAKKYOKU(I015)',\n        'FN-AEON_GROUP(JUSCOJ4)(C005)',\n        'FN-COSMOS_YAKUHIN(W030)',\n        'FN-COSTCO(A062)',\n        'FN-TSURUHA_HOKK+TOHO(E030)',\n        'FN-DONKIHOTE(A094)',\n        'FN-SUN_DRUG(D005)',\n        'FN-TGN',\n        'FN-KYORINDO(H011)',\n        'FN-LADY_YAKKYOKU(K067)',\n        'FN-KUSURINO FUKUTARO (G043)',\n        'FN-ELEVEN(L002)',\n        'FN-UNY(B009)',\n        'FN-SUNDAY(E015)'\n      ) THEN 'Div 1'\n      -- D-Com 顧客（org2 = D-Com）\n      WHEN org2 = 'D-Com' AND customer IN (\n        'FN-AMAZON JAPAN',\n        'FN-RAKUTEN'\n      ) THEN 'D-Com'\n      ELSE NULL\n    END AS division,          -- Div 1 / D-Com\n    customer,\n    cnos,\n    cnos_ya,\n    cnos_target\n  FROM term_cnos\n  WHERE customer IN (\n    -- Div 1 顧客\n    'FN-WELCIA YAKKYOKU',\n    'FN-CAINZ(C003)',\n    'FN-MATSUKIYO_COCOKARA (B008)',\n    'FN-SUGI_YAKKYOKU(I015)',\n    'FN-AEON_GROUP(JUSCOJ4)(C005)',\n    'FN-COSMOS_YAKUHIN(W030)',\n    'FN-COSTCO(A062)',\n    'FN-TSURUHA_HOKK+TOHO(E030)',\n    'FN-DONKIHOTE(A094)',\n    'FN-SUN_DRUG(D005)',\n    'FN-TGN',\n    'FN-KYORINDO(H011)',\n    'FN-LADY_YAKKYOKU(K067)',\n    'FN-KUSURINO FUKUTARO (G043)',\n    'FN-ELEVEN(L002)',\n    'FN-UNY(B009)',\n    'FN-SUNDAY(E015)',\n    -- D-Com 顧客\n    'FN-AMAZON JAPAN',\n    'FN-RAKUTEN'\n  )\n),\n\ncust_agg AS (\n  -- ▼term × division × customer ごとに C-NOS を集計（顧客粒度）\n  SELECT\n    term,\n    division,\n    customer,\n    SUM(cnos)        AS sum_cnos,\n    SUM(cnos_ya)     AS sum_cnos_ya,\n    SUM(cnos_target) AS sum_cnos_target\n  FROM filtered\n  WHERE division IS NOT NULL\n  GROUP BY\n    term,\n    division,\n    customer\n),\n\ndiv_agg AS (\n  -- ▼Div 1 / D-Com の合計行（17社合計 / 2社合計）\n  SELECT\n    term,\n    division,\n    SUM(sum_cnos)        AS sum_cnos,\n    SUM(sum_cnos_ya)     AS sum_cnos_ya,\n    SUM(sum_cnos_target) AS sum_cnos_target\n  FROM cust_agg\n  GROUP BY\n    term,\n    division\n),\n\nlabeled AS (\n  -- ▼Div 行と顧客行を「同じ列（customer_label）」に並べる\n  SELECT\n    term,\n    -- Div 1・D-Com の合計行\n    CASE\n      WHEN division = 'Div 1' THEN 'Div 1'\n      WHEN division = 'D-Com' THEN 'D-Com'\n      ELSE division\n    END AS customer_label,\n    sum_cnos,\n    sum_cnos_ya,\n    sum_cnos_target\n  FROM div_agg\n\n  UNION ALL\n\n  SELECT\n    term,\n    -- 顧客行：短い名前に変換（インデント用に先頭半角スペース）\n    CASE customer\n      WHEN 'FN-WELCIA YAKKYOKU'                 THEN ' Welcia'\n      WHEN 'FN-CAINZ(C003)'                     THEN ' Cainz'\n      WHEN 'FN-MATSUKIYO_COCOKARA (B008)'       THEN ' MCC'\n      WHEN 'FN-SUGI_YAKKYOKU(I015)'             THEN ' Sugi'\n      WHEN 'FN-AEON_GROUP(JUSCOJ4)(C005)'       THEN ' Aeon'\n      WHEN 'FN-COSMOS_YAKUHIN(W030)'            THEN ' Cosmos'\n      WHEN 'FN-COSTCO(A062)'                    THEN ' Costco'\n      WHEN 'FN-TSURUHA_HOKK+TOHO(E030)'         THEN ' Tsuruha'\n      WHEN 'FN-DONKIHOTE(A094)'                 THEN ' Donki'\n      WHEN 'FN-SUN_DRUG(D005)'                  THEN ' Sundrug'\n      WHEN 'FN-TGN'                             THEN ' TGN'\n      WHEN 'FN-KYORINDO(H011)'                  THEN ' Kyorindo'\n      WHEN 'FN-LADY_YAKKYOKU(K067)'             THEN ' Lady'\n      WHEN 'FN-KUSURINO FUKUTARO (G043)'        THEN ' Fukutaro'\n      WHEN 'FN-ELEVEN(L002)'                    THEN ' Eleven'\n      WHEN 'FN-UNY(B009)'                       THEN ' Uny'\n      WHEN 'FN-SUNDAY(E015)'                    THEN ' Sunday'\n      WHEN 'FN-AMAZON JAPAN'                    THEN ' Amazon'\n      WHEN 'FN-RAKUTEN'                         THEN ' Rakuten'\n      ELSE customer\n    END AS customer_label,\n    sum_cnos,\n    sum_cnos_ya,\n    sum_cnos_target\n  FROM cust_agg\n)\n\n-- ▼最終出力：customer_label 1列で、Div1/顧客/D-Com/顧客を縦に並べる\nSELECT\n  term,\n  customer_label AS customer,\n  -- C-NOS IYA：今年 ÷ 昨年 × 100（小数第1位）\n  ROUND(\n    CASE\n      WHEN sum_cnos_ya = 0 THEN NULL\n      ELSE 100.0 * sum_cnos / sum_cnos_ya\n    END,\n    1\n  ) AS cnos_iya,\n  -- C-NOS $MM：cnos / 150.1 / 1000 / 1000（小数第1位）\n  ROUND(\n    sum_cnos / 150.1 / 1000.0 / 1000.0,\n    1\n  ) AS cnos_mm,\n  -- C-NOS Target $MM：target / 150.1 / 1000 / 1000（小数第1位）\n  ROUND(\n    CASE\n      WHEN sum_cnos_target = 0 THEN NULL\n      ELSE sum_cnos_target / 150.1 / 1000.0 / 1000.0\n    END,\n    1\n  ) AS cnos_target_mm,\n  -- vs Target %（ギャップ）：100 - (C-NOS / Target × 100)、小数第1位\n  ROUND(\n    CASE\n      WHEN sum_cnos_target = 0 THEN NULL\n      ELSE 100.0 - (100.0 * sum_cnos / sum_cnos_target)\n    END,\n    1\n  ) AS cnos_vs_target_pct\nFROM labeled\nORDER BY\n  term,\n  CASE\n    -- 表示順：Div 1 → Div 1 顧客 → D-Com → D-Com 顧客\n    WHEN customer = 'Div 1'    THEN 1\n    WHEN customer = ' Welcia'  THEN 2\n    WHEN customer = ' Cainz'   THEN 3\n    WHEN customer = ' MCC'     THEN 4\n    WHEN customer = ' Sugi'    THEN 5\n    WHEN customer = ' Aeon'    THEN 6\n    WHEN customer = ' Cosmos'  THEN 7\n    WHEN customer = ' Costco'  THEN 8\n    WHEN customer = ' Tsuruha' THEN 9\n    WHEN customer = ' Donki'   THEN 10\n    WHEN customer = ' Sundrug' THEN 11\n    WHEN customer = ' TGN'     THEN 12\n    WHEN customer = ' Kyorindo'THEN 13\n    WHEN customer = ' Lady'    THEN 14\n    WHEN customer = ' Fukutaro'THEN 15\n    WHEN customer = ' Eleven'  THEN 16\n    WHEN customer = ' Uny'     THEN 17\n    WHEN customer = ' Sunday'  THEN 18\n    WHEN customer = 'D-Com'    THEN 19\n    WHEN customer = ' Amazon'  THEN 20\n    WHEN customer = ' Rakuten' THEN 21\n    ELSE 99\n  END,\n  customer;",
          "reference": "Present a rich interactive HTML table with all boarders. The order of the table should be the same as provided. Make the table font 0.75 times smaller\nAvailable data: {}",
          "number_of_plots": 0
        }
      ]
    }
  ],
  "agent_version": "v1",
  "system_prompts": {
    "section_writer_prompt": "\nYou are an agent who is an expert business report writer for Procter & Gamble, specializing in data analysis.\nPlease keep going until the user’s query is completely resolved, before ending your turn and yielding back to the user.Only terminate your turn when you are sure that the problem is solved.\n\n# Task\nWrite the '{section_title}' section of a business report.\n\n# Context\n`{section_description}`\n\n# Data and Instructions from user\n- Outline a specific, simple, and verifiable sequence of steps to address what the user wants to achieve in this section.\n- Here is the data from the user:\n`{items}`\n\n# Output Format\n- Strictly {report_format} format\n- Begin with the section title as a level-2 heading\n- Follow all formatting rules for tables, lists, graph, etc.\n\n# Requirements\n1. Use only the provided data - DO NOT introduce external facts or data\n2. Analyze metrics explaining business meaning without mentioning raw column names\n3. Ensure all insights directly connect to the underlying data\n\n# Instructions\n- If you receive any IMAGE_<plotid> placeholders at the end of the instructions, include them EXACTLY as returned by the tool\n- Do not try to render the IMAGE_<plotid> yourself, they will be handled later\n- Do not create or modify any IMAGE_<plotid> references yourself\n- Never wrap IMAGE_<plotid> in any HTML or markdown tags, paste them as they are\n\n# Key Terms\n- P1M: Past 1 month (most recent)\n- P3M: Past 3 months\n- P6M: Past 6 months\n- P12M: Past 12 months\n- IYA: Index Year Ago (current ÷ last year x 100)\n- CCC: Customer Category Combination\n- $MM: 1,000,000 USD.\n\n# Process\n1. First, plan your section structure based on the data\n2. Write section content following {report_format} conventions\n3. Verify all IMAGE_plotid references match exactly with tool responses\n",
    "report_compiler_prompt": "You are an expert business report writer.\nYour task is to compile the following sections into a single,\ncohesive business report.\n\n## Sections\n{sections}\n\n## Instructions\n- Output must be strictly in {report_format} format.\n- Begin with a level 1 heading for the report title.\n- Present each section in the order provided, using their {report_format} headings\n- If you receive an executive summary section:\n    * Present the Executive Summary first, then each section in the order provided, using their {report_format} headings.\n    * If you receive more than one summary sections, combine them into **a single section** titled Executive Summary, combining their insights in a concise manner.\n- Do not alter the content of each section, except for adding brief transitions or a summary if needed for clarity.\n- Ensure logical flow and smooth transitions.\n- Do not add or omit any information.\n- IMPORTANT: If you receive any image placeholders in the format in the format IMAGE_<plotid>, where a valid <plotid> is a random 8-character alphanumeric identifier,\n    it must be included verbatim in the report to be replaced with the actual image later. Not in any other format (i.e. IMAGE_plotID, IMAGE_1, etc.) as they may have been hallucinated.\n- IMPORTANT 2: \n- Preserve all image placeholders related to plots exactly as they appear in the tool response, thye will be rendered later. Ensure the report is in 70% of the viewport width, don't consume the full viewable screen. \n- Have the report title 'C-NOS Overview on Oct 2025' h1 tag, centerlined with solid #0037b1 color line below every section name. \n\n**Remember: Output must be strictly in {report_format} format.**\n**Preserve all **valid** `IMAGE_plotid` tags verbatim.**",
    "report_format_checker_prompt": "You are a meticulous business report editor for Procter & Gamble, specializing in data analysis reports.\n\nYour primary task is to review and correct both summary sentences and HTML tables in {report_format} business reports.\n\n## Review Criteria\n\n### Summary Sentences Verification:\n\nThe report should start with formatted sentences (no additional text before them):\n\n**Check for:**\n✓ **No Extra Content**: No introductory paragraphs, explanations, or conclusions before the table\n✓ **Correct Formatting**: Proper use of color coding (green for positive, red for negative), underline, and bold styling\n✓ **Proper Values**: Percentages formatted as integers with %, numeric values formatted appropriately\n✓ **HTML Formatting**: Proper use of `<span>` tags with inline styles for colored/formatted text\n\n### General Formatting:\n- Readability and clarity\n- Consistent {report_format} formatting (headings, tables, lists)\n- Professional tone and language\n- Correct use of {report_format} syntax\n- Ensure the report is in 70% of the viewport width, don't consume the full viewable screen\n\n### HTML Table Verification Checklist:\n\nWhen reviewing HTML tables, verify the following:\n\n✓ **Column Count Accuracy**: Count data columns in a data row = count of bottom-level header cells (accounting for colspan)\n✓ **No Empty Columns**: Every column must have data - remove any empty columns\n✓ **No Extra Columns**: No extra columns after the last defined group (table should end at the rightmost group)\n✓ **No Orphaned Columns**: No orphaned columns that don't belong to any header group\n✓ **Header Alignment**: All header cells have proper colspan/rowspan with no gaps in the header grid\n✓ **Header-Data Alignment**: Headers are properly aligned above their corresponding data columns\n✓ **Column Count Math**: Total column count matches the expected structure from the data\n✓ **Styling Applied**: All styling is present (backgrounds, borders, text alignment)\n✓ **Border Rendering**: Table renders correctly with border-collapse and proper spacing\n✓ **Conditional Formatting**: Conditional formatting is applied to the correct columns only\n✓ **Semantic HTML**: Proper use of <thead>, <tbody>, <th>, <td> elements\n✓ **Colspan/Rowspan Validity**: Sum of colspan in any header row equals total data columns\n\n### Specific Table Issues to Fix:\n\n1. **Header Misalignment**: Ensure headers with colspan span exactly the right number of columns beneath them\n2. **Extra Columns at End**: Remove any columns that appear after the defined structure ends\n3. **Empty Columns**: Delete columns that have no data in any row\n4. **Missing rowspan**: Add rowspan attributes where headers should span multiple rows (e.g., row header should span all header rows)\n5. **Incorrect colspan**: Recalculate and fix colspan values to match the actual number of columns they should cover\n6. **Styling Issues**: Apply proper backgrounds (#E7E6E6 for headers, #D9E1F2 for totals, #C6E0B4/#F8CBAD for conditional formatting)\n7. **Border Issues**: Ensure prominent borders between major groups and proper border-collapse\n8. **Pattern Recognition**: Recognize column naming patterns to identify when columns are complete (no subdivision needed)\n\n## Report to Review:\n`{report}`\n\n## Instructions\n- If the report is well-formatted, return the improved version with any minor corrections\n- If there are HTML table issues, fix them according to the checklist above\n- Ensure all tables have proper alignment and headers\n- Check that metrics are explained without using column names\n- Do not add new content or remove existing information\n- Do not skip, summarize or alter existing content\n- Output must be strictly in {report_format} format\n\n**Remember: Output must be strictly in {report_format} format, and do not output any other comments other than the report.**\n",
    "executive_summary_prompt": "\nYou are a senior business analyst at Procter & Gamble.\nYour mission: create a concise, senior-executive-ready Executive Summary that synthesizes key findings from the report sections below, highlighting trends, risks, and actionable opportunities.\n\nContext:\n{sections}\n\nOutput Requirements:\n- Format: strictly {report_format}\n- Length: **2000-3000 words**\n- Audience: senior execs — **clear, actionable, jargon-free**\n- Begin with a short opening paragraph summarizing overall performance\n- Include a bulleted list of **3-6 bullet points** that:\n  * Synthesize insights across multiple report sections\n  * Highlight top trends, emerging risks, and strategic opportunities\n  * Rephrase—**avoid verbatim sentences**\n- Use content **only** from the provided sections—**no new data**\n- If an “Executive” or “Topline Summary” section already exists, **build on it** and enrich with cross-section analysis\n- **Do not add any other headings or closing remarks**; start with: an Executive Summary h2 heading.\n"
  },
  "node_models": {
    "check_format": "GPT-41-2025-04-14",
    "compile_report": "GPT-41-2025-04-14",
    "plan_sections": "GPT-41-2025-04-14",
    "generate_plots": "GPT-41-2025-04-14"
  },
  "parameters": {}
}